<a href="https://colab.research.google.com/github/RyanBadai/Job-Scraping/blob/main/job_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [33]:
url_jobstreet = f'https://id.jobstreet.com/id/Data-Analyst-jobs?page={1}&sortmode=ListedDate'
page = s.get(url_jobstreet)
soup = BeautifulSoup(page.text, "lxml")
joblist = soup.find('div', class_='gepq850 eihuid5h eihuid53')

print(joblist)

<div class="gepq850 eihuid5h eihuid53"><a class="gepq850 gepq85f gepq850 gepq85f k5mille k5millg" data-automation="jobTitle" data-run-click-only="true" data-testid="job-card-title" href="/id/job/82166756?type=standard&amp;ref=search-standalone&amp;origin=cardTitle" id="job-title-82166756" rel="nofollow" target="_self">System Analyst</a></div>


In [34]:
req = requests.get(url_jobstreet)
req

<Response [200]>

In [ ]:
# Fungsi untuk scraping halaman utama
def scrape_main_page():
    jobs_data = []
    for i in range(1, 5):  # Scrape halaman 1-4
        url = f'https://id.jobstreet.com/id/Data-Analyst-jobs?page={i}&sortmode=ListedDate'
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
        }
        s = requests.session()
        s.headers.update(headers)
        try:
            page = s.get(url)
            soup = BeautifulSoup(page.text, "lxml")
            joblist = soup.find('div', class_='gepq850 _21bfxf1')
            if joblist is None:
                print(f"Job list not found on page {i}. Check the selector.")
                continue  # Skip ke halaman berikutnya
            for artikel in joblist.find_all('article', {'class': 'gepq850'}):
                posisi = artikel.find('div', {'class': 'gepq850 eihuid5h eihuid53'})
                perusahaan = artikel.find(attrs={'data-automation': 'jobCompany'})
                lokasi = artikel.find(attrs={'data-automation': 'jobLocation'})
                job_url = artikel.find('a', href=True)
                posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
                perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak disebutkan"
                lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
                job_url_text = 'https://id.jobstreet.com' + job_url['href'] if job_url and job_url['href'].startswith('/') else job_url['href'] if job_url else "URL tidak ditemukan"

                jobs_data.append({
                    "Posisi": posisi_text,
                    "Perusahaan": perusahaan_text,
                    "Lokasi": lokasi_text,
                    "URL": job_url_text
                })
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
    return jobs_data

# Fungsi untuk scraping halaman detail
def scrape_detail_page(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }
    try:
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "lxml")

        # Ekstrak deskripsi pekerjaan
        description_div = soup.find('div', {'data-automation': 'jobAdDetails'})
        description_text = description_div.get_text(strip=True) if description_div else "Deskripsi tidak ditemukan"

        return description_text
    except requests.exceptions.RequestException as e:
        print(f"Error scraping detail page {url}: {e}")
        return "Deskripsi tidak ditemukan"

# Main program
if __name__ == "__main__":
    # Step 1: Scrape halaman utama
    jobs_data = scrape_main_page()

    # Step 2: Scrape halaman detail untuk setiap URL
    for job in jobs_data:
        print(f"Scraping detail for: {job['Posisi']} ({job['URL']})")
        job['Deskripsi'] = scrape_detail_page(job['URL'])

    # Step 3: Simpan hasil ke CSV
    df = pd.DataFrame(jobs_data)
    df.to_csv("detailed_jobs_data.csv", index=False, encoding="utf-8")
    print("Data saved to detailed_jobs_data.csv")

Scraping detail for: Data Scientist (https://id.jobstreet.com/id/job/82136640?type=standard&ref=search-standalone)
Scraping detail for: Claim Analyst (https://id.jobstreet.com/id/job/82136443?type=standard&ref=search-standalone)
Scraping detail for: Sales Analyst & Operation Support (https://id.jobstreet.com/id/job/82136066?type=standard&ref=search-standalone)
Scraping detail for: Business Specialist (https://id.jobstreet.com/id/job/82135994?type=standard&ref=search-standalone)
Scraping detail for: Pricing Analyst (https://id.jobstreet.com/id/job/82135044?type=standard&ref=search-standalone)
Scraping detail for: Senior Treasury and Loan Channeling Operations (https://id.jobstreet.com/id/job/82134613?type=standard&ref=search-standalone)
Scraping detail for: Database Developer & Analyst (https://id.jobstreet.com/id/job/82134364?type=standard&ref=search-standalone)
Scraping detail for: Account Executive (Research & Consulting Industry) (https://id.jobstreet.com/id/job/82133455?type=standa

In [ ]:
df.head()

,Posisi,Perusahaan,Lokasi,URL,Deskripsi
0,Data Scientist,PT IDStar Cipta Teknologi,Jakarta Selatan,https://id.jobstreet.com/id/job/82136640?type=...,ResponsibilitiesConducting ambitious projects ...
1,Claim Analyst,Pengiklan Anonim,Jakarta Selatan,https://id.jobstreet.com/id/job/82136443?type=...,Kualifikasi:Usia maks. 35 tahunMin. D3 Perawat...
2,Sales Analyst & Operation Support,PT Eka Mas Republik,Jakarta Raya,https://id.jobstreet.com/id/job/82136066?type=...,Becoming a leading multimedia business in fibe...
3,Business Specialist,ENO INTERNATIONAL TRADING INC.,Jakarta Raya,https://id.jobstreet.com/id/job/82135994?type=...,What you'll be doing1. Flexibly use various re...
4,Pricing Analyst,PT KEX NEX EXPRESS,Jakarta Timur,https://id.jobstreet.com/id/job/82135044?type=...,"Requirements :Bachelor’s Degree in Economics, ..."


In [ ]:
jobs_data = []

for i in range(1,5):
  url = f'https://id.jobstreet.com/id/Data-Analyst-jobs?page={i}&sortmode=ListedDate'

  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

  s = requests.session()
  s.headers.update(headers)

  try:
      page = s.get(url)
      soup = BeautifulSoup(page.text, "lxml")
      joblist = soup.find('div', class_='gepq850 _21bfxf1')

      if joblist is None:
            print(f"Job list not found on page {i}. Check the selector.")
            continue # Skip to the next page

      for artikel in joblist.find_all('article', {'class': 'gepq850'}):
          posisi = artikel.find('div', {'class' : 'gepq850 eihuid5h eihuid53'})
          perusahaan = artikel.find(attrs={'data-automation':'jobCompany'})
          lokasi = artikel.find(attrs={'data-automation':'jobLocation'})
          gaji = artikel.find('span', {'class':'_1fggenz0 _1feq2e74z _1feq2e7r _474bdu0 _474bdu1 _474bdu21 v7shb4 _474bdu7 _9ci8fv0'})
          waktu = artikel.find('span',{'class':'_1fggenz0 _1feq2e74z _474bdu0 _474bdu1 _474bdu22 v7shb4 _474bdu7'})
          job_url = artikel.find('a', href=True)

          posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
          perusahaan_text = perusahaan.get_text() if perusahaan else "Perusahaan tidak disebutkan"
          lokasi_text = lokasi.get_text() if lokasi else "Lokasi tidak ditemukan"
          gaji_text = gaji.get_text() if gaji else "Gaji tidak disebutkan"
          waktu_text = waktu.get_text() if waktu else "Waktu tidak ditemukan"
          job_url_text = 'https://id.jobstreet.com' + job_url['href'] if job_url and job_url['href'].startswith('/') else job_url['href'] if job_url else "URL tidak ditemukan"

          jobs_data.append({
            "Posisi": posisi_text,
            "Perusahaan": perusahaan_text,
            "Lokasi": lokasi_text,
            "Gaji": gaji_text,
            "Waktu": waktu_text,
            "URL": job_url_text
          })

          print(posisi_text)
          print(job_url_text)
          # print(perusahaan_text)
          # print(lokasi_text)
          # print(gaji_text)
          # print(waktu_text)
          print("===============================")
  except requests.exceptions.RequestException as e:
      print(f"Error: {e}")

Data Scientist
https://id.jobstreet.com/id/job/82136640?type=standard&ref=search-standalone
Claim Analyst
https://id.jobstreet.com/id/job/82136443?type=standard&ref=search-standalone
Sales Analyst & Operation Support
https://id.jobstreet.com/id/job/82136066?type=standard&ref=search-standalone
Business Specialist
https://id.jobstreet.com/id/job/82135994?type=standard&ref=search-standalone
Pricing Analyst
https://id.jobstreet.com/id/job/82135044?type=standard&ref=search-standalone
Senior Treasury and Loan Channeling Operations
https://id.jobstreet.com/id/job/82134613?type=standard&ref=search-standalone
Database Developer & Analyst
https://id.jobstreet.com/id/job/82134364?type=standard&ref=search-standalone
Account Executive (Research & Consulting Industry)
https://id.jobstreet.com/id/job/82133455?type=standard&ref=search-standalone
Scientific Consultant
https://id.jobstreet.com/id/job/82126962?type=standard&ref=search-standalone
Staf Operasional Pangkalan Data, Penjaminan Mutu & Akredita

In [ ]:
df_jobs_full = pd.DataFrame(jobs_data)

df_jobs_full.head()

,Posisi,Perusahaan,Lokasi,Gaji,Waktu,URL
0,Data Scientist,PT IDStar Cipta Teknologi,Jakarta Selatan,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82136640?type=...
1,Claim Analyst,Pengiklan Anonim,Jakarta Selatan,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82136443?type=...
2,Sales Analyst & Operation Support,PT Eka Mas Republik,Jakarta Raya,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82136066?type=...
3,Business Specialist,ENO INTERNATIONAL TRADING INC.,Jakarta Raya,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82135994?type=...
4,Pricing Analyst,PT KEX NEX EXPRESS,Jakarta Timur,Gaji tidak disebutkan,Waktu tidak ditemukan,https://id.jobstreet.com/id/job/82135044?type=...


In [ ]:
df_jobs_full.shape

(128, 6)

# Glints

In [6]:
url = f'https://glints.com/vn/en/opportunities/jobs/explore?keyword=digital+marketing&country=ID&locationName=All+Cities%2FProvinces&page=2'
s = requests.session()
s.headers.update(headers)
page = s.get(url)
soup = BeautifulSoup(page.text, "lxml")
joblist = soup.find('div', class_='JobCardsc__JobcardContainer-sc-hmqj50-0 iirqVR CompactOpportunityCardsc__CompactJobCardWrapper-sc-dkg8my-5 hRilQl')

print(joblist)

None


In [3]:
jobs_data = []

for i in range(1,5):
  url = f'https://glints.com/vn/en/opportunities/jobs/explore?keyword=digital+marketing&country=ID&locationName=All+Cities%2FProvinces&page={i}'

  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.90"
    }

  s = requests.session()
  s.headers.update(headers)

  try:
      page = s.get(url)
      soup = BeautifulSoup(page.text, "lxml")
      joblist = soup.find('div', class_='MainContainersc__MainBody-sc-xmyke8-2 drnzBQ')

      if joblist is None:
            print(f"Job list not found on page {i}. Check the selector.")
            continue # Skip to the next page

      for artikel in joblist.select('div', {'class': 'JobCardsc__JobCardWrapper-sc-hmqj50-1 fnsbDp'}):
          posisi = artikel.find('div', {'class' : 'CompactOpportunityCardsc__JobTitle-sc-dkg8my-11 gwOhhF'})
          link = artikel.find('div', {'class' : 'CompactOpportunityCardsc__CompactJobCard-sc-dkg8my-4 ipGhBs job-search-results_job-card_link'})

          posisi_text = posisi.get_text() if posisi else "Posisi tidak ditemukan"
          link_text = link.get_text() if posisi else "Posisi tidak ditemukan"

          jobs_data.append({
            "Posisi": posisi_text,
            "Link": link_text
          })

          print(posisi_text)
          print(link_text)
          # print(perusahaan_text)
          # print(lokasi_text)
          # print(gaji_text)
          # print(waktu_text)
          print("===============================")
  except requests.exceptions.RequestException as e:
      print(f"Error: {e}")

Job list not found on page 1. Check the selector.
Job list not found on page 2. Check the selector.
Job list not found on page 3. Check the selector.
Job list not found on page 4. Check the selector.


# Testing

In [24]:
import requests
from bs4 import BeautifulSoup
import csv

In [30]:
url_glints = "https://glints.com/vn/en/opportunities/jobs/explore?keyword=digital+marketing&country=ID&locationName=All+Cities%2FProvinces"

In [31]:
req = requests.get(url_glints)
req

<Response [403]>

In [27]:
soup = BeautifulSoup(req.text, "html.parser")

In [28]:
semua = soup.find_all('div', class_='ExploreTabsc__Body-sc-1fr7yeh-3 bFDfCF')
print(semua)

[]
